In [1]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [4]:
columns = read_json('../../orderdListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


## Load data

In [16]:
true_df = pd.read_csv('../../Image_20genres/poster2genreMat.csv')
true_df

,filename,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,47metersdown.jpg,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,adogsjourneymovie.jpg,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,aftermathmovie.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,aftermovie.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,alitamovie.jpg,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,angelhasfallen.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,angrybirdsmovie.jpg,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,annabellemovie.jpg,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
8,annamovie.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,apollo11movie.jpg,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [17]:
pred_df = pd.read_csv('./input/resnet50_trainable_2e_poster_bi.csv')
pred_df

,username,acc,truth,top_n,Action,Adventure,Animation,Biography,Comedy,Crime,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,47metersdown.jpg,0.333333,"['Adventure', 'Drama', 'Horror']","['Adventure', 'Action', 'Fantasy']",1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,adogsjourneymovie.jpg,1.000000,"['Adventure', 'Comedy', 'Drama']","['Drama', 'Comedy', 'Adventure']",0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,aftermathmovie.jpg,0.333333,"['Drama', 'Romance', 'War']","['Drama', 'Adventure', 'Biography']",0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,aftermovie.jpg,0.500000,"['Drama', 'Romance']","['Adventure', 'Drama']",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,alitamovie.jpg,1.000000,"['Action', 'Adventure', 'Sci-Fi']","['Action', 'Adventure', 'Sci-Fi']",1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,angelhasfallen.jpg,0.500000,"['Action', 'Thriller']","['Adventure', 'Action']",1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,angrybirdsmovie.jpg,1.000000,"['Adventure', 'Animation', 'Comedy']","['Animation', 'Adventure', 'Comedy']",0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,annabellemovie.jpg,0.000000,"['Horror', 'Mystery', 'Thriller']","['Adventure', 'Fantasy', 'Action']",1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,annamovie.jpg,0.000000,"['Action', 'Thriller']","['Adventure', 'Horror']",0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,apollo11movie.jpg,0.500000,"['Documentary', 'History']","['Documentary', 'Horror']",0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
y_true = np.array(true_df.iloc[:, 1:])
y_pred = np.array(pred_df.iloc[:, 4:])
print(y_true.shape, y_pred.shape)

(97, 20) (97, 20)


In [19]:
cmatrix = multilabel_confusion_matrix(y_true, y_pred)
cmatrix

array([[[52, 22],
        [ 3, 20]],

       [[30, 37],
        [ 0, 30]],

       [[83,  5],
        [ 2,  7]],

       [[88,  2],
        [ 3,  4]],

       [[52,  9],
        [ 7, 29]],

       [[84,  2],
        [ 7,  4]],

       [[96,  0],
        [ 0,  1]],

       [[48,  4],
        [16, 29]],

       [[94,  0],
        [ 1,  2]],

       [[83,  5],
        [ 4,  5]],

       [[94,  0],
        [ 3,  0]],

       [[79,  3],
        [11,  4]],

       [[91,  1],
        [ 2,  3]],

       [[81,  3],
        [ 9,  4]],

       [[86,  1],
        [10,  0]],

       [[83,  5],
        [ 3,  6]],

       [[93,  0],
        [ 3,  1]],

       [[81,  0],
        [13,  3]],

       [[95,  0],
        [ 1,  1]],

       [[96,  0],
        [ 1,  0]]], dtype=int64)

In [20]:
cmatrix.shape

(20, 2, 2)

In [23]:
conmat = pd.DataFrame(data = {
    'genre' : columns,
    'TP' : cmatrix[:, 1, 1],
    'FP' : cmatrix[:, 0, 1],
    'FN' : cmatrix[:, 1, 0],
    'TN' : cmatrix[:, 0, 0]
}, columns=['genre', 'TP', 'FP', 'FN', 'TN'])
conmat

,genre,TP,FP,FN,TN
0,Action,20,22,3,52
1,Adventure,30,37,0,30
2,Animation,7,5,2,83
3,Biography,4,2,3,88
4,Comedy,29,9,7,52
5,Crime,4,2,7,84
6,Documentary,1,0,0,96
7,Drama,29,4,16,48
8,Family,2,0,1,94
9,Fantasy,5,5,4,83


In [14]:
conmat.to_csv('./confusion_matrix.csv', index = 0)